In [270]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict


import sys
import ENVIRONMENT_VARIABLES as EV

In [312]:
teams = EV.TEAMS
variables =["Year", "Win", "Defense", "Attack", "Margin", "Home", "Versus",  "Round", "Halftime_Attack", "Halftime_Defense", "Tries", "FirstHalf_Tries", "SecondHalf_Tries", "FirstHalf_Tries_Other", "SecondHalf_Tries_Other"]
years =  [2021]
select_year = years[0]

In [309]:

# Initialize an empty dictionary to store data for each year
years_arr = {}

# Open the JSON file containing NRL data
with open(f'./data/nrl_data_{years[0]}.json', 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    
    # Extract NRL data from the loaded JSON
    data = data['NRL']
    
    # Iterate over each year in the years list
    for year in years:
        # Extract data for the current year and store it in the years_arr dictionary
        # Note: years.index(year) returns the index of the current year in the years list
        #       This index is then used to access the corresponding data for that year
        years_arr[year] = data[years.index(year)][str(year)]


In [313]:
round_game_data = defaultdict(dict)

with open(f'./data/nrl_detailed_match_data_{select_year}.json', 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    data = data['NRL']
    for round in range(0, 27-1):
        round_data = data[round]
        for key, value in round_data.items():
            game_datas = round_data[key]
            for game in game_datas:
                # Now the game 
                game_name = list(game.keys())[0]
                round_game_data[round][game_name] = game[game_name]

In [ ]:
elements = round_game_data[2]['Broncos v Dragons']['away']['try_minutes']
print(len(elements))
count_greater_than_40 = sum(int(element.replace("'", "")) <= 40 for element in elements)
count_greater_than_40

3


3

In [ ]:

# Create a DataFrame with columns representing combinations of team and variable names
df = pd.DataFrame(columns=[f"{team} {variable}" for team in teams for variable in variables])

# Initialize an empty list to store data for all rounds
all_store = []

# Iterate over each year in the years list
for year in years:
    # Iterate over each round (assuming 26 rounds)
    for round in range(0, 27):
        try:
            # Extract data for the current round
            round_data = years_arr[year][round][str(round+1)]
            
            # Create an empty feature array 
            round_store = np.zeros([len(teams)*len(variables)], dtype=int)
            round_teams = []
            
            # Iterate over each game in the round data
            for game in round_data:
                # Extract information about the game
                h_team = game['Home']
                h_score = int(game['Home_Score'])
                a_team = game['Away']
                a_score = int(game['Away_Score'])
                
                h_team_name = h_team.replace(" ", "-")
                a_team_name = a_team.replace(" ", "-")
                

                rgd = round_game_data[round][f"{h_team} v {a_team}"]
                match_data_away = rgd['away']['half_time']
                match_data_home = rgd['home']['half_time']
                
                try_amount_home = rgd['home']['try_minutes']
                try_second_half_home = sum(int(element.replace("'", ""))  > 40 for element in try_amount_home)
                try_amount_home = len(try_amount_home)
                try_first_half_home = try_amount_home-try_second_half_home
                
                try_amount_away = rgd['away']['try_minutes']
                try_second_half_away = sum(int(e.replace("'", ""))  > 40 for e in try_amount_away)
                try_amount_away = len(try_amount_away)
                try_first_half_away = try_amount_away-try_second_half_away
                


                
                # Determine win or lose for each team
                h_team_win, a_team_win = h_score >= a_score, a_score >= h_score
                
                # Determine home team status
                h_home, a_home = 1, 0
                
                # Determine versus index
                h_versus, a_versus= teams.index(a_team), teams.index(h_team)
                
                # Determine defense (points let in)
                h_team_defense = a_score
                a_team_defense = h_score  
                
                # Determine attack points scored
                h_team_attack = h_score 
                a_team_attack = a_score   
                
                # Determine margin
                h_team_margin =  h_score - a_score   
                a_team_margin =  a_score - h_score        
                
                # Keep track of which teams played to work out which teams had a bye 
                round_teams.append(h_team)
                round_teams.append(a_team)
                
                # Find the index of the team in the overarching array 
                a_team_idx = teams.index(a_team)
                h_team_idx = teams.index(h_team)
                
                # Determine feature map index
                a_team_idx_fm = a_team_idx * len(variables)
                h_team_idx_fm = h_team_idx * len(variables)
                
                # Populate data for away team
                for idx, data in zip(range(a_team_idx_fm, a_team_idx_fm + len(variables)), 
                                    [year, a_team_win, a_team_defense, a_team_attack, a_team_margin, a_home, a_versus, round + 1, match_data_away, match_data_home,
                                     try_amount_away, try_first_half_away, try_second_half_away, try_first_half_home, try_second_half_home]):
                    round_store[idx] = data

                # Populate data for home team
                for idx, data in zip(range(h_team_idx_fm, h_team_idx_fm + len(variables)),
                                    [year, h_team_win, h_team_defense, h_team_attack, h_team_margin, h_home, h_versus, round + 1, match_data_home, match_data_away,
                                     try_amount_home, try_first_half_home, try_second_half_home, try_first_half_away, try_second_half_away]):
                    round_store[idx] = data
                    
                # if game['Details'] == 'Broncos vs Dragons':
                #     print(f"{[year, h_team_win, h_team_defense, h_team_attack, h_team_margin, h_home, h_versus, round + 1, match_data_home, match_data_away, try_amount_home, try_first_half_home, try_second_half_home]}")
                #     print(f"{[year, a_team_win, a_team_defense, a_team_attack, a_team_margin, a_home, a_versus, round + 1, match_data_away, match_data_home, try_amount_away, try_first_half_away, try_second_half_away]}")
                #     print(rgd['away']['try_minutes'])
                
            # Determine teams with a bye
            bye_teams = list(set(teams) - set(round_teams))
            
            # Assign values for teams with a bye
            for bye_team in bye_teams:
                b_team_idx = teams.index(bye_team)
                b_team_idx_fm = b_team_idx * len(variables)
                round_store[b_team_idx_fm] = year
                round_store[b_team_idx_fm+1] = -1
                round_store[b_team_idx_fm+2] = -1
                round_store[b_team_idx_fm+3] = -1
                round_store[b_team_idx_fm+4] = 0
                round_store[b_team_idx_fm+5] = -1
                round_store[b_team_idx_fm+6] = -1
                round_store[b_team_idx_fm+7] = round+1
                round_store[b_team_idx_fm+8] = -1
                round_store[b_team_idx_fm+9] = -1
                round_store[b_team_idx_fm+10] = -1
                round_store[b_team_idx_fm+11] = -1
                round_store[b_team_idx_fm+12] = -1
                round_store[b_team_idx_fm+13] = -1
                
                
            # Append the round data to the all_store list
            all_store.append(round_store)
            
            # Add the new row to the DataFrame using loc
            df.loc[len(df)] = round_store
        except Exception as ex:
            print(ex)
        

'Panthers v Sea Eagles'
'Storm v Rabbitohs'
'Dragons v Sharks'
'Titans v Wests Tigers'
'Knights v Sea Eagles'
'Raiders v Cowboys'
'Sharks v Sea Eagles'
'Broncos v Sharks'
'Rabbitohs v Broncos'
'Bulldogs v Knights'
'Knights v Broncos'
'Storm v Sea Eagles'
'Titans v Cowboys'
'Cowboys v Dragons'
'Sea Eagles v Cowboys'
'Sea Eagles v Storm'
'Sharks v Storm'
'Cowboys v Sharks'
'Eels v Broncos'
'Sea Eagles v Roosters'
'Roosters v Broncos'
'Panthers v Storm'
'Rabbitohs v Panthers'
'Panthers v Warriors'
'Eels v Storm'
'Storm v Raiders'
'Eels v Raiders'


In [274]:
filtered_columns = [col for col in df.columns if 'halftime_attack' in col.lower() or 'halftime_defense' in col.lower()or 'attack' in col.lower() or 'defense' in col.lower() or 'firsthalf_tries' in col.lower()or 'secondhalf_tries' in col.lower() or 'tries' in col.lower() or 'secondhalf_tries_other' in col.lower() or 'firsthalf_tries_other' in col.lower()]
filtered_df = df[filtered_columns]

In [275]:
for team in EV.TEAMS:
    # Calculate the difference between Attack and Halftime columns
    attack_col = f"{team} Attack"
    attack_def_col = f"{team} Defense"
    halftime_col = f"{team} Halftime_Attack"
    halftime_def_col = f"{team} Halftime_Defense"
    second_half_col = f"{team} SecondHalf_Attack"
    second_half_def_col = f"{team} SecondHalf_Attack"
    
    second_tries_col = f"{team} SecondHalf_Tries"
    second_tries_def_col = f"{team} SecondHalf_Tries"
    first_tries_col = f"{team} First_Tries"
    first_tries_def_col = f"{team} First_Tries"
    
    filtered_df[second_half_col] = filtered_df[attack_col] - filtered_df[halftime_col]
    filtered_df[second_half_def_col] = filtered_df[attack_def_col] - filtered_df[halftime_def_col]

filtered_df = filtered_df.reindex(sorted(filtered_df.columns), axis=1)


In [276]:
filtered_df

,Broncos Attack,Broncos Defense,Broncos FirstHalf_Tries,Broncos FirstHalf_Tries_Other,Broncos Halftime_Attack,Broncos Halftime_Defense,Broncos SecondHalf_Attack,Broncos SecondHalf_Tries,Broncos SecondHalf_Tries_Other,Broncos Tries,...,Wests Tigers Attack,Wests Tigers Defense,Wests Tigers FirstHalf_Tries,Wests Tigers FirstHalf_Tries_Other,Wests Tigers Halftime_Attack,Wests Tigers Halftime_Defense,Wests Tigers SecondHalf_Attack,Wests Tigers SecondHalf_Tries,Wests Tigers SecondHalf_Tries_Other,Wests Tigers Tries


In [279]:

# Initialize an empty list to store data for all rounds
all_store = []
match_first_halves = [] 
match_second_halves= [] 
match_first_halves_teams = defaultdict(list)
match_second_halves_teams = defaultdict(list)
win_both_halfs = 0




# Iterate over each year in the years list
for year in years:
    # Iterate over each round (assuming 26 rounds)
    for round in range(0, 27):
        try:
            # Extract data for the current round
            round_data = years_arr[year][round][str(round+1)]
            
            # Create an empty feature array 
            round_store = np.zeros([len(teams)*len(variables)], dtype=int)
            round_teams = []
            
            # Iterate over each game in the round data
            for game in round_data:
                # Extract information about the game
                h_team = game['Home']
                h_score = int(game['Home_Score'])
                a_team = game['Away']
                a_score = int(game['Away_Score'])
                
                h_team_name = h_team.replace(" ", "-")
                a_team_name = a_team.replace(" ", "-")
                

                rgd = round_game_data[round][f"{h_team} v {a_team}"]
                match_data_away = rgd['away']['half_time']
                match_data_home = rgd['home']['half_time']
                
                match_first_halves_teams[h_team].append(int(match_data_home))
                match_first_halves_teams[a_team].append(int(match_data_away))
                
                match_second_halves_teams[h_team].append(int(h_score)-int(match_data_home))
                match_second_halves_teams[a_team].append(a_score-int(match_data_away))
                
                
                
                match_half = int(match_data_away)+int(match_data_home)
                match_first_halves.append(match_half)
                
                second_half = h_score-int(match_data_home)+a_score-int(match_data_away)
                match_second_halves.append(second_half)
                
                match_data_home = int(match_data_home)
                match_data_away = int(match_data_away)

                
                if (match_data_home>match_data_away &  (h_score-int(match_data_home)>a_score-int(match_data_away))) or (match_data_home<match_data_away &  (h_score-int(match_data_home)<a_score-int(match_data_away))) :
                    win_both_halfs += 1 
                
                
                
        except Exception as ex:
            print(ex)

'Panthers v Sea Eagles'
'Storm v Rabbitohs'
'Dragons v Sharks'
'Titans v Wests Tigers'
'Knights v Sea Eagles'
'Raiders v Cowboys'
'Sharks v Sea Eagles'
'Broncos v Sharks'
'Rabbitohs v Broncos'
'Bulldogs v Knights'
'Knights v Broncos'
'Storm v Sea Eagles'
'Titans v Cowboys'
'Cowboys v Dragons'
'Dragons v Rabbitohs'
'Sea Eagles v Storm'
'Sharks v Storm'
'Cowboys v Sharks'
'Eels v Broncos'
'Sea Eagles v Roosters'
'Roosters v Broncos'
'Panthers v Storm'
'Rabbitohs v Panthers'
'Broncos v Eels'
'Eels v Storm'
'Panthers v Eels'
'Eels v Raiders'


In [278]:
win_both_halfs*100/len(match_first_halves)

ZeroDivisionError: division by zero

In [ ]:
first_half_team_sum = {}
second_half_team_sum = {}


for key, arr  in  match_first_halves_teams.items():
    arr = np.array(arr)
    first_half_team_sum[key] = arr.mean()
    
for key, arr  in  match_second_halves_teams.items():
    arr = np.array(arr)
    second_half_team_sum[key] = arr.mean()
     
first_half_team_sum

{'Dragons': 12.0,
 'Rabbitohs': 8.0,
 'Broncos': 24.0,
 'Eels': 11.0,
 'Panthers': 4.0}

In [ ]:
for (k1, mfh), (k2, msh) in zip(match_first_halves_teams.items(), match_second_halves_teams.items()):
    result = [1 if mfh[i] > msh[i] else (0 if mfh[i] == msh[i] else 2) for i in range(len(mfh))]

    tie = result.count(0)
    first = result.count(1)
    second = result.count(2)
    print(f"Team: {k1} Tie: {tie} 1st: {first} 2nd: {second}")

Team: Dragons Tie: 0 1st: 0 2nd: 1
Team: Rabbitohs Tie: 0 1st: 1 2nd: 0
Team: Broncos Tie: 0 1st: 1 2nd: 0
Team: Eels Tie: 0 1st: 1 2nd: 1
Team: Panthers Tie: 0 1st: 0 2nd: 1


In [ ]:
second_half_team_sum

{'Dragons': 20.0,
 'Rabbitohs': 4.0,
 'Broncos': -18.0,
 'Eels': 19.5,
 'Panthers': 23.0}

In [ ]:
for (z, i), j in zip(first_half_team_sum.items(), second_half_team_sum.values()):
    print(f"{z}: greater 1st half: {i>j} {i-j}")

Dragons: greater 1st half: False -8.0
Rabbitohs: greater 1st half: True 4.0
Broncos: greater 1st half: True 42.0
Eels: greater 1st half: False -8.5
Panthers: greater 1st half: False -19.0


In [ ]:

total_count = len(match_first_halves)
total_count_same = 0 
for first_half, second_half in zip(match_first_halves,match_second_halves):
    if first_half == second_half:
        total_count_same +=1 

print(f"{total_count_same}/{total_count} or {total_count_same*100/total_count:.2f}%")

0/3 or 0.00%


In [ ]:
# Now do the same thing for tries 

In [ ]:
same_tries_o  = 0
dif_tries_o  = 0
metrics = [] 

print(f"{'Team':>10} {'Metric':>10} {'Avg Tries':>10} {'1st Tries':>10} {'2nd Tries':>10} {'Diff Tries':>10} {'Same Tries':>10} {'Different Tries':>10}")
for team in EV.TEAMS:
    
    # Calculate the difference between Attack and Halftime columns
    second_tries_col = f"{team} SecondHalf_Tries"
    first_tries_col = f"{team} FirstHalf_Tries"
    
    second_tries_col_other = f"{team} SecondHalf_Tries_Other"
    first_tries_col_other = f"{team} FirstHalf_Tries_Other"
    

        
    second_sum = filtered_df[second_tries_col] + filtered_df[second_tries_col_other]
    first_sum = filtered_df[first_tries_col_other]+filtered_df[first_tries_col]


    same_tries_o += (second_sum ==first_sum).sum()

    dif_tries_o  += (second_sum !=first_sum).sum()
    
    tries= f"{team} Tries"
    team = team.replace("Wests Tigers", "Tigers")
    same_tries  = (filtered_df[first_tries_col] == filtered_df[second_tries_col]).sum()
    not_same_tries = (filtered_df[first_tries_col] != filtered_df[second_tries_col]).sum()
    
    
    metric = abs(filtered_df[first_tries_col].mean() - filtered_df[second_tries_col].mean())*(not_same_tries-same_tries)/not_same_tries
    metrics.append(metric)
    # Data
    print(f"{team:>10} {metric:>10.2f} {filtered_df[tries].mean():>10.2f} {filtered_df[first_tries_col].mean():>10.2f} {filtered_df[second_tries_col].mean():>10.2f}  {abs(filtered_df[second_tries_col].mean()-filtered_df[first_tries_col].mean()):>10.2f} {same_tries:>10.2f}{not_same_tries:>10.2f}")

    
print(f"{same_tries_o }\t{dif_tries_o }")


      Team     Metric  Avg Tries  1st Tries  2nd Tries Diff Tries Same Tries Different Tries
   Broncos        nan        nan        nan        nan         nan       0.00      0.00
  Roosters        nan        nan        nan        nan         nan       0.00      0.00
    Tigers        nan        nan        nan        nan         nan       0.00      0.00
 Rabbitohs        nan        nan        nan        nan         nan       0.00      0.00
     Storm        nan        nan        nan        nan         nan       0.00      0.00
      Eels        nan        nan        nan        nan         nan       0.00      0.00
   Raiders        nan        nan        nan        nan         nan       0.00      0.00
   Knights        nan        nan        nan        nan         nan       0.00      0.00
   Dragons        nan        nan        nan        nan         nan       0.00      0.00
Sea Eagles        nan        nan        nan        nan         nan       0.00      0.00
  Panthers        nan      

In [ ]:
from itertools import combinations
pairs = combinations(metrics, 2)

# Calculate the product for each pair
products = [pair[1] + pair[0] for pair in pairs]
sorted_products = sorted(products, reverse=False)
sorted_products

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [ ]:
team_1 = "Eels"
team_1_post = EV.TEAMS.index(team_1)

team_2 = "Sea Eagles"
team_2_post = EV.TEAMS.index(team_2)

metric = metrics[team_1_post] + metrics[team_2_post]
rank = sorted_products.index(metric)


print(f"{rank} of {len(sorted_products)} {rank*100/len(sorted_products):.2f}%")
print(f"{metrics[team_1_post]:.2f} {metrics[team_2_post]:.2f} ")


ValueError: nan is not in list

In [ ]:
EV.TEAMS

['Broncos',
 'Roosters',
 'Wests Tigers',
 'Rabbitohs',
 'Storm',
 'Eels',
 'Raiders',
 'Knights',
 'Dragons',
 'Sea Eagles',
 'Panthers',
 'Sharks',
 'Bulldogs',
 'Dolphins',
 'Titans',
 'Cowboys',
 'Warriors']

## Highest Scoring Half

In [280]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict


import sys
import ENVIRONMENT_VARIABLES as EV

In [317]:
teams = EV.TEAMS
variables =["Year", "Win", "Defense", "Attack", "Margin", "Home", "Versus",  "Round", "Halftime_Attack", "Halftime_Defense", "Tries", "FirstHalf_Tries", "SecondHalf_Tries"]
years =  [2024]

In [318]:

# Initialize an empty dictionary to store data for each year
years_arr = {}

# Open the JSON file containing NRL data
with open(f"./data/nrl_data_{years[0]}.json", 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    
    # Extract NRL data from the loaded JSON
    data = data['NRL']
    
    # Iterate over each year in the years list
    for year in years:
        # Extract data for the current year and store it in the years_arr dictionary
        # Note: years.index(year) returns the index of the current year in the years list
        #       This index is then used to access the corresponding data for that year
        years_arr[year] = data[years.index(year)][str(year)]


In [292]:
data

[{'2022': [{'1': [{'Details': 'Panthers vs Sea Eagles',
      'Date': 'Thursday 10th March',
      'Home': 'Panthers',
      'Home_Score': '28',
      'Away': 'Sea Eagles',
      'Away_Score': '6',
      'Venue': 'BlueBet Stadium, Penrith'},
     {'Details': 'Raiders vs Sharks',
      'Date': 'Friday 11th March',
      'Home': 'Raiders',
      'Home_Score': '24',
      'Away': 'Sharks',
      'Away_Score': '19',
      'Venue': 'GIO Stadium, Canberra'},
     {'Details': 'Broncos vs Rabbitohs',
      'Date': 'Friday 11th March',
      'Home': 'Broncos',
      'Home_Score': '11',
      'Away': 'Rabbitohs',
      'Away_Score': '4',
      'Venue': 'Suncorp Stadium, Brisbane'},
     {'Details': 'Roosters vs Knights',
      'Date': 'Saturday 12th March',
      'Home': 'Roosters',
      'Home_Score': '6',
      'Away': 'Knights',
      'Away_Score': '20',
      'Venue': 'Sydney Cricket Ground, Sydney'},
     {'Details': 'Warriors vs Dragons',
      'Date': 'Saturday 12th March',
      'Home': 

In [321]:
round_game_data = defaultdict(dict)

with open(f'./data/nrl_detailed_match_data_{years[0]}.json', 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    data = data['NRL']
    for round in range(0, 27-1):
        round_data = data[round]
        for key, value in round_data.items():
            game_datas = round_data[key]
            for game in game_datas:
                # Now the game 
                game_name = list(game.keys())[0]
                round_game_data[round][game_name] = game[game_name]

In [295]:

# Create a DataFrame with columns representing combinations of team and variable names
df = pd.DataFrame(columns=[f"{team} {variable}" for team in teams for variable in variables])

# Initialize an empty list to store data for all rounds
all_store = []

# Iterate over each year in the years list
for year in years:
    # Iterate over each round (assuming 26 rounds)
    for round in range(0, 27):
        try:
            # Extract data for the current round
            round_data = years_arr[year][round][str(round+1)]
            
            # Create an empty feature array 
            round_store = np.zeros([len(teams)*len(variables)], dtype=int)
            round_teams = []
            
            # Iterate over each game in the round data
            for game in round_data:
                # Extract information about the game
                h_team = game['Home']
                h_score = int(game['Home_Score'])
                a_team = game['Away']
                a_score = int(game['Away_Score'])
                
                h_team_name = h_team.replace(" ", "-")
                a_team_name = a_team.replace(" ", "-")
                

                rgd = round_game_data[round][f"{h_team} v {a_team}"]
                match_data_away = rgd['away']['half_time']
                match_data_home = rgd['home']['half_time']
                
                try_amount_home = rgd['home']['try_minutes']
                try_second_half_home = sum(int(element.replace("'", ""))  > 40 for element in try_amount_home)
                try_amount_home = len(try_amount_home)
                try_first_half_home = try_amount_home-try_second_half_home
                
                try_amount_away = rgd['away']['try_minutes']
                try_second_half_away = sum(int(e.replace("'", ""))  > 40 for e in try_amount_away)
                try_amount_away = len(try_amount_away)
                try_first_half_away = try_amount_away-try_second_half_away
                


                
                # Determine win or lose for each team
                h_team_win, a_team_win = h_score >= a_score, a_score >= h_score
                
                # Determine home team status
                h_home, a_home = 1, 0
                
                # Determine versus index
                h_versus, a_versus= teams.index(a_team), teams.index(h_team)
                
                # Determine defense (points let in)
                h_team_defense = a_score
                a_team_defense = h_score  
                
                # Determine attack points scored
                h_team_attack = h_score 
                a_team_attack = a_score   
                
                # Determine margin
                h_team_margin =  h_score - a_score   
                a_team_margin =  a_score - h_score        
                
                # Keep track of which teams played to work out which teams had a bye 
                round_teams.append(h_team)
                round_teams.append(a_team)
                
                # Find the index of the team in the overarching array 
                a_team_idx = teams.index(a_team)
                h_team_idx = teams.index(h_team)
                
                # Determine feature map index
                a_team_idx_fm = a_team_idx * len(variables)
                h_team_idx_fm = h_team_idx * len(variables)
                
                # Populate data for away team
                for idx, data in zip(range(a_team_idx_fm, a_team_idx_fm + len(variables)), 
                                    [year, a_team_win, a_team_defense, a_team_attack, a_team_margin, a_home, a_versus, round + 1, match_data_away, match_data_home, try_amount_away, try_first_half_away, try_second_half_away]):
                    round_store[idx] = data

                # Populate data for home team
                for idx, data in zip(range(h_team_idx_fm, h_team_idx_fm + len(variables)),
                                    [year, h_team_win, h_team_defense, h_team_attack, h_team_margin, h_home, h_versus, round + 1, match_data_home, match_data_away, try_amount_home, try_first_half_home, try_second_half_home]):
                    round_store[idx] = data
                    
                # if game['Details'] == 'Broncos vs Dragons':
                #     print(f"{[year, h_team_win, h_team_defense, h_team_attack, h_team_margin, h_home, h_versus, round + 1, match_data_home, match_data_away, try_amount_home, try_first_half_home, try_second_half_home]}")
                #     print(f"{[year, a_team_win, a_team_defense, a_team_attack, a_team_margin, a_home, a_versus, round + 1, match_data_away, match_data_home, try_amount_away, try_first_half_away, try_second_half_away]}")
                #     print(rgd['away']['try_minutes'])
                
            # Determine teams with a bye
            bye_teams = list(set(teams) - set(round_teams))
            
            # Assign values for teams with a bye
            for bye_team in bye_teams:
                b_team_idx = teams.index(bye_team)
                b_team_idx_fm = b_team_idx * len(variables)
                round_store[b_team_idx_fm] = year
                round_store[b_team_idx_fm+1] = -1
                round_store[b_team_idx_fm+2] = -1
                round_store[b_team_idx_fm+3] = -1
                round_store[b_team_idx_fm+4] = 0
                round_store[b_team_idx_fm+5] = -1
                round_store[b_team_idx_fm+6] = -1
                round_store[b_team_idx_fm+7] = round+1
                round_store[b_team_idx_fm+8] = -1
                round_store[b_team_idx_fm+9] = -1
                round_store[b_team_idx_fm+10] = -1
                round_store[b_team_idx_fm+11] = -1
                
            # Append the round data to the all_store list
            all_store.append(round_store)
            
            # Add the new row to the DataFrame using loc
            df.loc[len(df)] = round_store
        except Exception as ex:
            print(ex)
        

'Eels v Raiders'


In [296]:
df

,Broncos Year,Broncos Win,Broncos Defense,Broncos Attack,Broncos Margin,Broncos Home,Broncos Versus,Broncos Round,Broncos Halftime_Attack,Broncos Halftime_Defense,...,Warriors Attack,Warriors Margin,Warriors Home,Warriors Versus,Warriors Round,Warriors Halftime_Attack,Warriors Halftime_Defense,Warriors Tries,Warriors FirstHalf_Tries,Warriors SecondHalf_Tries
0,2022,1,4,11,7,1,3,1,6,4,...,16,-12,1,8,1,10,14,3,2,1
1,2022,1,10,16,6,0,12,2,4,4,...,18,-2,0,14,2,18,16,3,3,0
2,2022,0,38,12,-26,1,15,3,6,14,...,16,4,0,2,3,14,12,2,2,0
3,2022,0,20,6,-14,0,16,4,6,16,...,20,14,1,0,4,16,6,3,3,0
4,2022,0,24,20,-4,1,1,5,10,0,...,25,1,1,15,5,12,18,4,2,2
5,2022,0,40,12,-28,0,10,6,6,12,...,14,-8,0,1,6,8,6,2,1,1
6,2022,1,14,34,20,1,12,7,6,10,...,10,-60,0,4,7,10,16,2,2,0
7,2022,1,7,16,9,1,11,8,6,7,...,21,1,1,6,8,12,20,3,2,1
8,2022,1,12,32,20,0,3,9,12,6,...,10,-19,0,11,9,0,1,2,2,0
9,2022,1,0,38,38,0,9,10,14,0,...,30,-2,1,3,10,6,26,5,1,4


In [297]:
filtered_columns = [col for col in df.columns if 'halftime_attack' in col.lower() or 'halftime_defense' in col.lower()or 'attack' in col.lower() or 'defense' in col.lower() or 'firsthalf_tries' in col.lower()or 'secondhalf_tries' in col.lower() or 'tries' in col.lower()]
filtered_df = df[filtered_columns]

In [298]:
for team in EV.TEAMS:
    # Calculate the difference between Attack and Halftime columns
    attack_col = f"{team} Attack"
    attack_def_col = f"{team} Defense"
    halftime_col = f"{team} Halftime_Attack"
    halftime_def_col = f"{team} Halftime_Defense"
    second_half_col = f"{team} SecondHalf_Attack"
    second_half_def_col = f"{team} SecondHalf_Attack"
    
    second_tries_col = f"{team} SecondHalf_Tries"
    second_tries_def_col = f"{team} SecondHalf_Tries"
    first_tries_col = f"{team} First_Tries"
    first_tries_def_col = f"{team} First_Tries"
    
    filtered_df[second_half_col] = filtered_df[attack_col] - filtered_df[halftime_col]
    filtered_df[second_half_def_col] = filtered_df[attack_def_col] - filtered_df[halftime_def_col]

filtered_df = filtered_df.reindex(sorted(filtered_df.columns), axis=1)


C:\Users\beauh\AppData\Local\Temp\ipykernel_21992\1796965856.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[second_half_col] = filtered_df[attack_col] - filtered_df[halftime_col]
C:\Users\beauh\AppData\Local\Temp\ipykernel_21992\1796965856.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[second_half_def_col] = filtered_df[attack_def_col] - filtered_df[halftime_def_col]
C:\Users\beauh\AppData\Local\Temp\ipykernel_21992\1796965856.py:15: SettingWithCopyWarning: 
A value is tr

In [ ]:

# Initialize an empty list to store data for all rounds
all_store = []
match_first_halves = [] 
match_second_halves= [] 
match_first_halves_teams = defaultdict(list)
match_second_halves_teams = defaultdict(list)
win_both_halfs = 0




# Iterate over each year in the years list
for year in years:
    # Iterate over each round (assuming 26 rounds)
    for round in range(0, 27):
        print(f'Analysing: {round}', end=', ')
        try:
            # Extract data for the current round
            round_data = years_arr[year][round][str(round+1)]
            
            # Create an empty feature array 
            round_store = np.zeros([len(teams)*len(variables)], dtype=int)
            round_teams = []
            
            # Iterate over each game in the round data
            for game in round_data:
                # Extract information about the game
                h_team = game['Home']
                h_score = int(game['Home_Score'])
                a_team = game['Away']
                a_score = int(game['Away_Score'])
                
                h_team_name, a_team_name = [team.replace(" ", "-") for team in [h_team, a_team]]

                rgd = round_game_data[round][f"{h_team} v {a_team}"]
                match_data_away = rgd['away']['half_time']
                match_data_home = rgd['home']['half_time']
                
                match_first_halves_teams[h_team].append(int(match_data_home))
                match_first_halves_teams[a_team].append(int(match_data_away))
                
                match_second_halves_teams[h_team].append(int(h_score)-int(match_data_home))
                match_second_halves_teams[a_team].append(a_score-int(match_data_away))
                
                
                
                match_half = int(match_data_away)+int(match_data_home)
                match_first_halves.append(match_half)
                
                second_half = h_score-int(match_data_home)+a_score-int(match_data_away)
                match_second_halves.append(second_half)
                
                match_data_home = int(match_data_home)
                match_data_away = int(match_data_away)

                
                if (match_data_home>match_data_away &  (h_score-int(match_data_home)>a_score-int(match_data_away))) or (match_data_home<match_data_away &  (h_score-int(match_data_home)<a_score-int(match_data_away))) :
                    win_both_halfs += 1 
                
                
                
        except Exception as ex:
            print(ex)

Analysing: 0, Analysing: 1, Analysing: 2, Analysing: 3, Analysing: 4, Analysing: 5, Analysing: 6, Analysing: 7, Analysing: 8, Analysing: 9, Analysing: 10, Analysing: 11, Analysing: 12, Analysing: 13, Analysing: 14, Analysing: 15, Analysing: 16, Analysing: 17, Analysing: 18, Analysing: 19, Analysing: 20, Analysing: 21, Analysing: 22, Analysing: 23, Analysing: 24, Analysing: 25, Analysing: 26, 'Broncos v Storm'


In [ ]:
first_half_team_sum = {}
second_half_team_sum = {}


for key, arr  in  match_first_halves_teams.items():
    arr = np.array(arr)
    first_half_team_sum[key] = arr.mean()
    
for key, arr  in  match_second_halves_teams.items():
    arr = np.array(arr)
    second_half_team_sum[key] = arr.mean()
     
first_half_team_sum

{'Eels': 12.5,
 'Storm': 14.08695652173913,
 'Warriors': 11.304347826086957,
 'Knights': 12.73913043478261,
 'Panthers': 14.043478260869565,
 'Broncos': 12.695652173913043,
 'Sea Eagles': 10.869565217391305,
 'Bulldogs': 8.826086956521738,
 'Cowboys': 12.652173913043478,
 'Raiders': 11.391304347826088,
 'Sharks': 13.130434782608695,
 'Rabbitohs': 9.217391304347826,
 'Dolphins': 9.652173913043478,
 'Roosters': 9.26086956521739,
 'Wests Tigers': 7.217391304347826,
 'Titans': 11.695652173913043,
 'Dragons': 9.73913043478261}

## Team Performance Based on Halfs (most tries())

In [ ]:
for (k1, mfh), (k2, msh) in zip(match_first_halves_teams.items(), match_second_halves_teams.items()):
    result = [1 if mfh[i] > msh[i] else (0 if mfh[i] == msh[i] else 2) for i in range(len(mfh))]

    tie = result.count(0)
    first = result.count(1)
    second = result.count(2)
    print(f"Team: {k1} Tie: {tie} 1st: {first} 2nd: {second}")

Team: Eels Tie: 2 1st: 10 2nd: 12
Team: Storm Tie: 2 1st: 12 2nd: 9
Team: Warriors Tie: 4 1st: 10 2nd: 9
Team: Knights Tie: 4 1st: 9 2nd: 10
Team: Panthers Tie: 5 1st: 11 2nd: 7
Team: Broncos Tie: 2 1st: 10 2nd: 11
Team: Sea Eagles Tie: 3 1st: 10 2nd: 10
Team: Bulldogs Tie: 4 1st: 9 2nd: 10
Team: Cowboys Tie: 0 1st: 15 2nd: 8
Team: Raiders Tie: 2 1st: 13 2nd: 8
Team: Sharks Tie: 4 1st: 12 2nd: 7
Team: Rabbitohs Tie: 0 1st: 6 2nd: 17
Team: Dolphins Tie: 2 1st: 6 2nd: 15
Team: Roosters Tie: 1 1st: 10 2nd: 12
Team: Wests Tigers Tie: 5 1st: 8 2nd: 10
Team: Titans Tie: 0 1st: 14 2nd: 9
Team: Dragons Tie: 2 1st: 9 2nd: 12


In [ ]:
for (z, i), j in zip(first_half_team_sum.items(), second_half_team_sum.values()):
    print(f"{z}: greater 1st half: {i>j} {i-j}")

Eels: greater 1st half: True 0.5416666666666661
Storm: greater 1st half: True 2.304347826086957
Warriors: greater 1st half: False -1.8260869565217384
Knights: greater 1st half: False -0.3478260869565215
Panthers: greater 1st half: True 1.9565217391304337
Broncos: greater 1st half: False -1.4347826086956523
Sea Eagles: greater 1st half: True 0.3913043478260878
Bulldogs: greater 1st half: False -0.08695652173913082
Cowboys: greater 1st half: True 2.086956521739131
Raiders: greater 1st half: True 1.913043478260871
Sharks: greater 1st half: True 0.39130434782608603
Rabbitohs: greater 1st half: False -5.565217391304348
Dolphins: greater 1st half: False -1.8260869565217384
Roosters: greater 1st half: False -0.8695652173913047
Wests Tigers: greater 1st half: False -1.7826086956521738
Titans: greater 1st half: True 1.9565217391304337
Dragons: greater 1st half: False -0.6086956521739122


## Highest Half

In [299]:

total_count = len(match_first_halves)
total_count_same = 0 
for first_half, second_half in zip(match_first_halves,match_second_halves):
    if first_half == second_half:
        total_count_same +=1 



print(f"Occurence of a tie is: {total_count_same}/{total_count} or {total_count_same*100/total_count:.2f}%")

ZeroDivisionError: division by zero

In [ ]:
print(f"First Half Average Points: {sum(match_first_halves)/len(match_first_halves)}\tSecond Half Average Points: {sum(match_second_halves)/len(match_second_halves)}")

First Half Average Points: 22.479591836734695	Second Half Average Points: 22.806122448979593


In [ ]:
greater_first_half = 0 
greater_second_half = 0 
for first_half, second_half in zip(match_first_halves,match_second_halves):
    if first_half > second_half:
        greater_first_half +=1 
    elif second_half > first_half:
        greater_second_half += 1
print(f"First Half Greater Points: {greater_first_half*100/total_count:.2f}%\nTied Points: {total_count_same*100/total_count:.2f}%\nSecond Half Greater Points: {greater_second_half*100/total_count:.2f}%")

first_half_win = greater_first_half*100/total_count
tie_win = total_count_same*100/total_count
second_half_win = greater_second_half*100/total_count



First Half Greater Points: 47.96%
Tied Points: 6.12%
Second Half Greater Points: 45.92%


## Betting on try scoring half

In [ ]:

# Create the slider widget
a_slider =2.10
b_slider = 2.18

Enter the odds of not occuring

In [ ]:
odds = total_count_same*100/total_count
print(f"Odds: {odds:.2f}%")

Odds: 6.12%


Enter the betting company values

In [ ]:
stake = 100
rounds_of_betting = 20
# Perform arbitage on the sliders 
ratio = b_slider/(b_slider+a_slider)
payout = ratio*stake*(a_slider)

# Return of investment % 
ROI =(payout-stake)*100/stake

print(f"ROI: {ROI:.2f}%")

ROI: 6.96%


In [ ]:
print(f"Win ${ROI/100*stake*rounds_of_betting}\nLose ${odds/100*stake*rounds_of_betting}\nTotal after {rounds_of_betting} rounds  @ {stake} = ${ROI/100*rounds_of_betting*stake-odds/100*stake*rounds_of_betting}")

Win $139.25233644859847
Lose $122.44897959183673
Total after 20 rounds  @ 100 = $16.803356856761738


# Win both halves

In [322]:

# Initialize an empty list to store data for all rounds
all_store = []
match_first_halves = [] 
match_second_halves= [] 
match_first_halves_teams = defaultdict(list)
match_second_halves_teams = defaultdict(list)
win_both_halfs = 0
does_not_win_both_halfs = 0

versues = []
first_half_scores_a = []
first_half_scores_h = []

second_half_scores_a = []
second_half_scores_h = []


wins_a = []
wins_h = []

dates = [] 
ladder = defaultdict(int) 


# Iterate over each year in the years list
for year in years:
    # Iterate over each round (assuming 26 rounds)
    for round in range(0, 27):
        print(f'Analysing: {round}', end=', ')
        try:
            # Extract data for the current round
            round_data = years_arr[year][round][str(round+1)]
            
            # Create an empty feature array 
            round_store = np.zeros([len(teams)*len(variables)], dtype=int)
            round_teams = []
            
            # Iterate over each game in the round data
            for game in round_data:
                versues.append(game['Details'])
                dates.append(game['Date'])
                # Extract information about the game
                h_team = game['Home']
                h_score = int(game['Home_Score'])
                a_team = game['Away']
                a_score = int(game['Away_Score'])
                
                h_team_name, a_team_name = [team.replace(" ", "-") for team in [h_team, a_team]]
                
                print('here')
                if round != 0:
                    wins_a.append(ladder[a_team_name])
                    wins_h.append(ladder[h_team_name])
                else:
                    wins_a.append(0)
                    wins_h.append(0)   
                print('there')    
                if h_score > a_score: 
                    ladder[h_team_name] += 1
                    ladder[a_team_name] += 0
                elif a_score> h_score:
                    ladder[a_team_name] += 1    
                    ladder[h_team_name] += 0
                


                rgd = round_game_data[round][f"{h_team} v {a_team}"]
                match_data_away = rgd['away']['half_time']
                match_data_home = rgd['home']['half_time']
                
                match_first_halves_teams[h_team].append(int(match_data_home))
                match_first_halves_teams[a_team].append(int(match_data_away))
                
                match_second_halves_teams[h_team].append(int(h_score)-int(match_data_home))
                match_second_halves_teams[a_team].append(a_score-int(match_data_away))
                print('heres')
                
                
                match_half = int(match_data_away)+int(match_data_home)
                match_first_halves.append(match_half)
                
                second_half = h_score-int(match_data_home)+a_score-int(match_data_away)
                match_second_halves.append(second_half)
                
                match_data_home = int(match_data_home)
                match_data_away = int(match_data_away)
                
                
                a_team_first_half_points = match_data_away
                h_team_first_half_points = match_data_home
                
                a_team_second_half_points = a_score-int(match_data_away)
                h_team_second_half_points = h_score-int(match_data_home)
                
                first_half_scores_a.append(a_team_first_half_points)
                first_half_scores_h.append(h_team_first_half_points)

                second_half_scores_a.append(a_team_second_half_points)
                second_half_scores_h.append(h_team_second_half_points)
                
                
                if((a_team_first_half_points > h_team_first_half_points) and (a_team_second_half_points > h_team_second_half_points)):
                    win_both_halfs += 1
                elif((a_team_first_half_points < h_team_first_half_points) and (a_team_second_half_points < h_team_second_half_points)):
                    win_both_halfs += 1
                else:
                    does_not_win_both_halfs += 1

                
                
        except Exception as ex:
            print(f"Error: {ex}")
            

Analysing: 0, here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
Analysing: 1, here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
Analysing: 2, here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
Analysing: 3, here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
Analysing: 4, here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
Analysing: 5, here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
Analysing: 6, here
there
heres
here
there
heres
here
there
heres
here
there
heres
here
there
heres
h

In [315]:
data = {
    'Versus': versues[:-1],
    'Dates': dates[:-1],
    'Win Away': wins_a[:-1],
    'Win Home': wins_h[:-1],
    'First Half Scores Away': first_half_scores_a,
    'First Half Scores Home': first_half_scores_h,
    'Second Half Scores Away': second_half_scores_a,
    'Second Half Scores Home': second_half_scores_h,

}


df_win_both_halfs = pd.DataFrame(data)

In [316]:
df_win_both_halfs.to_csv('calum_3.csv')

In [303]:
print(f"Wins Both Halfs: {win_both_halfs}\nDoes Not Win Both: {does_not_win_both_halfs}\nWins Both Halfs: {win_both_halfs*100/(does_not_win_both_halfs+win_both_halfs):.2f}%\nDoes not Win Both Halfs: {does_not_win_both_halfs*100/(does_not_win_both_halfs+win_both_halfs):.2f}%")

Wins Both Halfs: 91
Does Not Win Both: 105
Wins Both Halfs: 46.43%
Does not Win Both Halfs: 53.57%


In [ ]:

# Create the slider widget
a_slider =4.5
b_slider = 3.25

In [ ]:
odds = does_not_win_both_halfs*100/(does_not_win_both_halfs+win_both_halfs)
print(f"Odds: {odds:.2f}%")

Odds: 55.10%


In [ ]:
stake = 100
rounds_of_betting = 20
# Perform arbitage on the sliders 
ratio = b_slider/(b_slider+a_slider)
payout = ratio*stake*(a_slider)

# Return of investment % 
ROI =(payout-stake)*100/stake

print(f"ROI: {ROI:.2f}%")

ROI: 88.71%


In [ ]:
print(f"Win ${ROI/100*stake*rounds_of_betting}\nLose ${odds/100*stake*rounds_of_betting}\nTotal after {rounds_of_betting} rounds  @ {stake} = ${ROI/100*rounds_of_betting*stake-odds/100*stake*rounds_of_betting}")

Win $1774.193548387097
Lose $1102.0408163265304
Total after 20 rounds  @ 100 = $672.1527320605664


# OTHER SHIT DO NOT ENTER THIS ZONE!

In [ ]:
import random

In [ ]:
matches = 26 * 8
odds_of_failure = 6.8
start = 500
win_amount = stake*ROI/100

balance = start

for _ in range(matches):
    if random.random() >= (odds_of_failure/100):
        balance += win_amount
        print('Win ', end="")
    else:
        balance -= stake
        print('Loss')

print("\nFinal Balance:", balance)


Win Win Win Win Win Win Win Win Win Win Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Loss
Win Win Win Win Win Win Win Win Win Win Win Loss
Win Win Loss
Win Win Win Win Win Win Win Win Win Loss
Win Win Win Win Win Win Loss
Win Win Win Win Win Win Win Loss
Loss
Win Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Loss
Win Win Win Win Win Win Win Win Win Win Win Loss
Win Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Loss
Win Win Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win 
Final Balance: -631.7073170731616


In [ ]:
filtered_df

,Broncos Attack,Broncos Defense,Broncos FirstHalf_Tries,Broncos Halftime_Attack,Broncos Halftime_Defense,Broncos SecondHalf_Attack,Broncos SecondHalf_Tries,Broncos Tries,Bulldogs Attack,Bulldogs Defense,...,Warriors SecondHalf_Tries,Warriors Tries,Wests Tigers Attack,Wests Tigers Defense,Wests Tigers FirstHalf_Tries,Wests Tigers Halftime_Attack,Wests Tigers Halftime_Defense,Wests Tigers SecondHalf_Attack,Wests Tigers SecondHalf_Tries,Wests Tigers Tries
0,13,12,2,12,6,6,0,2,6,31,...,2,4,10,22,0,2,12,10,2,2
1,28,16,2,10,10,6,3,5,26,12,...,1,2,12,14,1,0,1,13,1,2
2,40,18,1,8,16,2,6,7,26,22,...,2,5,22,26,1,6,20,6,3,4
3,18,12,1,4,0,12,3,4,14,16,...,2,3,12,24,1,6,22,2,1,2
4,46,12,5,28,0,12,3,8,15,14,...,3,5,12,46,0,0,28,18,2,2
5,14,20,1,4,14,6,2,3,16,50,...,3,4,22,28,1,6,18,10,3,4
6,43,26,2,10,14,12,5,7,4,30,...,2,4,-1,-1,-1,-1,-1,0,0,-1
7,26,16,4,22,6,10,0,4,20,33,...,1,4,16,22,1,6,12,10,2,3
8,6,32,1,6,12,20,0,1,18,16,...,0,0,12,8,1,8,8,0,1,2
9,32,6,4,20,0,6,2,6,30,34,...,0,1,18,16,2,14,10,6,1,3


# Other shit 

In [ ]:
print(f"First Half: {sum(match_first_halves)/len(match_first_halves)}\tSecond Half: {sum(match_second_halves)/len(match_second_halves)}")

First Half: 22.479591836734695	Second Half: 22.806122448979593


In [ ]:
greater_first_half = 0 
greater_second_half = 0 
for first_half, second_half in zip(match_first_halves,match_second_halves):
    if first_half > second_half:
        greater_first_half +=1 
    elif second_half > first_half:
        greater_second_half += 1
print(f"{greater_first_half*100/total_count:.2f}%\n{total_count_same*100/total_count:.2f}%\n{greater_second_half*100/total_count:.2f}%")

first_half_win = greater_first_half*100/total_count
tie_win = total_count_same*100/total_count
second_half_win = greater_second_half*100/total_count



47.96%
6.12%
45.92%


In [ ]:
bet = 1
bet_a = 2.1
bet_b = 2.00
bet_c = 13
bet_amount = 100 

winnings = bet_a*first_half_win/100*bet_amount-1*bet_amount




print(winnings)

0.7142857142857082


# Gambling

In [ ]:

# Create the slider widget
a_slider =2.10
b_slider = 2.00

In [ ]:
stake = 100
rounds_of_betting = 20
ratio = b_slider/(b_slider+a_slider)
payout = ratio*stake*(a_slider)
ROI =(payout-stake)*100/stake

ROI

2.4390243902439295

In [ ]:
odds = total_count_same*100/total_count
odds

6.122448979591836

In [ ]:
rounds_of_betting

20

In [ ]:
odds < ROI


print(f"Win ${ROI*rounds_of_betting}\nLose ${odds*rounds_of_betting}\nTotal after {rounds_of_betting} rounds  @ {stake} = ${ROI*rounds_of_betting-odds*rounds_of_betting}")

Win $48.78048780487859
Lose $122.44897959183673
Total after 20 rounds  @ 100 = $-73.66849178695814


In [ ]:
import random

In [ ]:
matches = 26 * 8
odds_of_failure = 6.8
start = 500
win_amount = stake*ROI/100

balance = start

for _ in range(matches):
    if random.random() >= (odds_of_failure/100):
        balance += win_amount
        print('Win ', end="")
    else:
        balance -= stake
        print('Loss')

print("\nFinal Balance:", balance)


Win Win Win Win Win Loss
Win Win Win Win Win Win Win Loss
Win Win Win Win Loss
Win Win Win Win Win Win Win Win Win Loss
Win Win Win Win Win Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Loss
Win Win Win Loss
Win Win Win Win Win Win Win Win Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Loss
Win Win Loss
Win Win Win Win Win Win Win Loss
Loss
Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win Win 
Final Balance: -426.8292682926756
